In [1]:
import pandas as pd
import re
from tqdm import tqdm
import torch
import torchmetrics
import ast
import random
import transformers
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
import tree
import functions
from functions import Dataset_create_embeddings_from_tokens_for_text, Clinical_Bert_Model, Dataset_create_embeddings_from_classdescripsion_for_classsentence_list
import sequence_label
import wandb
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import multiprocessing as mp
import argparse_para

/u/home/niur/venv_1/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/u/home/niur/venv_1/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
args = argparse_para.arg_parser()

In [3]:
tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease = tree.build_a_tree()
# build the tree, it is four dicts.

100%|██████████| 1391/1391 [00:00<00:00, 74420.59it/s]


In [4]:

tokenizer = transformers.BertTokenizer.from_pretrained('./Model_Bio_Clinical_DS_BERT/Bio_DischargeSummary_BERT/hub/models--emilyalsentzer--Bio_Discharge_Summary_BERT/snapshots/affde836a50e4d333f15dae9270f5a856d59540b', ignore_mismatched_sizes=True)
clinical_DS_bert = transformers.BertModel.from_pretrained('./Model_Bio_Clinical_DS_BERT/Bio_DischargeSummary_BERT/hub/models--emilyalsentzer--Bio_Discharge_Summary_BERT/snapshots/affde836a50e4d333f15dae9270f5a856d59540b')
# get the pre-trained bert model.   

Some weights of the model checkpoint at ./Model_Bio_Clinical_DS_BERT/Bio_DischargeSummary_BERT/hub/models--emilyalsentzer--Bio_Discharge_Summary_BERT/snapshots/affde836a50e4d333f15dae9270f5a856d59540b were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('GPU', torch.cuda.is_available())

clinical_Bert_Model = Clinical_Bert_Model(clinical_DS_bert=clinical_DS_bert)
clinical_Bert_Model = clinical_Bert_Model.to(device)

GPU True


In [6]:
# this dataframe contains the incd9 classes embeddings

df_classes_descripsion = functions.create_classes_descripsions(tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease)
# df_classes_descripsion_embeddings has two columns: 'classes' and 'descripsion', and the length of it is all the category and subclass(end tokens not included).

class_input_ids_tensor = functions.create_class_bert_tokens(df_classes_descripsion, tokenizer)  # , class_token_type_idstensor, class_attention_mask_tensor
class_input_ids_tensor = class_input_ids_tensor.detach()
create_class_embeddings_dataset = Dataset_create_embeddings_from_classdescripsion_for_classsentence_list(class_input_ids_tensor)  # , class_token_type_idstensor, class_attention_mask_tensor
create_class_embeddings_dataloader = DataLoader(create_class_embeddings_dataset, 1, shuffle=False, drop_last=False, num_workers=8)
class_embeddings_tensor = functions.create_classes_embeddings(create_class_embeddings_dataloader, clinical_Bert_Model, device)
print('the shape of class_embedding_tensor is: ', class_embeddings_tensor.shape)
class_embeddings_dict = functions.create_class_embeddings_dict(list(df_classes_descripsion['classes']), class_embeddings_tensor)
print('the length of class embedding is(end tokens not included)',len(class_embeddings_dict))
# keys are classes and values are embeddings tensors, and the length of it is all the category and subclass(end tokens not included).
# the shape of embeddings is (n, 768)


 11%|█         | 1683/15978 [00:17<02:27, 97.14it/s]


KeyboardInterrupt: 

In [ ]:
df_train_top500 = pd.read_csv('./data/top500_datasets/trainset_top500_new.csv')
df_validation_top500 = pd.read_csv('./data/top500_datasets/validationset_top500_new.csv')

In [ ]:
df_train_top500 = functions.text_cleaning(df_train_top500)
df_validation_top500 = functions.text_cleaning(df_validation_top500)
df_train_top500 = functions.convert_icdstr_to_list(df_train_top500)
df_validation_top500 = functions.convert_icdstr_to_list(df_validation_top500)
df_train_top500 = sequence_label.create_sequence_label_depth_first(tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease, df_train_top500)
df_validation_top500 = sequence_label.create_sequence_label_depth_first(tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease, df_validation_top500)
print(df_train_top500.columns)
print(df_train_top500.shape, df_validation_top500.shape)


/u/home/niur/htc_mimic3/functions.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['TEXT'].iloc[i] = re.sub("(Admission Date:)|(Discharge Date:)|(Service:)|(Date of Birth:)|(Sex:)|(Attending:)|(Provider:)|(Name:)|(Date/Time:)|(MD Phone:)|(Completed by:)|(Job#:)|(Dictated By:)", "", dataframe['TEXT'].iloc[i])
/u/home/niur/htc_mimic3/functions.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['TEXT'].iloc[i] = re.sub(r'\[\*\*.*?\*\*\]', '', dataframe['TEXT'].iloc[i])  # delete [**      **]
/u/home/niur/htc_mimic3/functions.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

Index(['HADM_ID', 'TEXT', 'ICD9_CODE', 'SEQUENCIAL_LABEL'], dtype='object')
(8496, 4) (1500, 4)


In [ ]:


input_ids_tensor_train = functions.create_text_tokens(df_train_top500, tokenizer) # , token_type_idstensor_train, attention_mask_tensor_train
input_ids_tensor_train = input_ids_tensor_train.detach()
print('train token shape:', input_ids_tensor_train.shape)
input_ids_tensor_validation = functions.create_text_tokens(df_validation_top500, tokenizer) # , token_type_idstensor_validation, attention_mask_tensor_validation
input_ids_tensor_validation = input_ids_tensor_validation.detach()
print('valid token shape:', input_ids_tensor_validation.shape)

# train_dataset_top500_create_text_embeddings = Dataset_create_embeddings_from_tokens_for_text(input_ids_tensor_train) # , token_type_idstensor_train, attention_mask_tensor_train
# validation_datasdet_top500_create_text_embeddings = Dataset_create_embeddings_from_tokens_for_text(input_ids_tensor_validation) #, token_type_idstensor_validation, attention_mask_tensor_validation
# train_dataloader_top500_create_text_embeddings = DataLoader(train_dataset_top500_create_text_embeddings, 1, shuffle=False, drop_last=False, num_workers=8)
# validation_dataloader_top500_create_text_embeddings = DataLoader(validation_datasdet_top500_create_text_embeddings, 1, shuffle=False, drop_last=False, num_workers=8)


# train_text_embeddings_tensor = functions.create_text_embeddings(train_dataloader_top500_create_text_embeddings, clinical_Bert_Model, device)
# validation_text_embeddings_tensor = functions.create_text_embeddings(validation_dataloader_top500_create_text_embeddings, clinical_Bert_Model, device)
# # this is the outputs of encoder and will be the cross attention input of decoder.
# print('train text embedding shape:', train_text_embeddings_tensor.shape)
# print('valid text embedding shape:', validation_text_embeddings_tensor.shape)

100%|██████████| 8496/8496 [02:31<00:00, 55.93it/s]


train token shape: torch.Size([8496, 4, 512])


100%|██████████| 1500/1500 [00:26<00:00, 55.66it/s]

valid token shape: torch.Size([1500, 4, 512])


In [ ]:
mask_dict = functions.create_tokens_mask_dict(tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease)
# it s a dictionary contains the -inf and 0 mask(1d tensor) of each category and subclass.
class_label_index_dict = functions.create_class_label_index_dict(mask_dict)
# {'start':0, .......}
index_to_class_label_dict = functions.create_index_to_class_label_dict(mask_dict)
# {0:'start', .......}
print('the length of mask dict is:',len(class_label_index_dict))

the length of mask dict is: 17392


In [ ]:
train_sequencial_label_list = list(df_train_top500['SEQUENCIAL_LABEL'])       # [[L], [L], [L]......]
valiation_sequencial_label_list = list(df_validation_top500['SEQUENCIAL_LABEL'])
# here the data pass into the dataset can not be tensors, because change into embeddings need the complete class embedding dict.

train_label_index_list = functions.convrt_sequencial_label_to_label_index(class_label_index_dict, train_sequencial_label_list)  # [tensor(L), tensor(L), tensor(L), tensor(L), tensor(L).......]
validation_label_index_list = functions.convrt_sequencial_label_to_label_index(class_label_index_dict, valiation_sequencial_label_list)

In [ ]:
class Dataset_transformer(Dataset):
    def __init__(self, input_ids_tensor_train, label_index_list):
        self.input_ids_tensor_train = input_ids_tensor_train   #(N, 4, 512)
        self.label_index_list = label_index_list      # [tensor(L), tensor(L), tensor(L), tensor(L), tensor(L).......]

    def __len__(self):
        return self.input_ids_tensor_train.shape[0]

    def __getitem__(self, index):
        return self.input_ids_tensor_train[index], self.label_index_list[index]
    
def collate_fn(batch):
    input_ids_tensor_train = torch.stack([item[0] for item in batch])   # (b,4,512)
    batch_label_index = torch.nn.utils.rnn.pad_sequence([item[1] for item in batch], batch_first=True, padding_value=17391)  # (b,L) L is the largest length in this batch
    max_L = batch_label_index.shape[1]
    padding_mask = []
    for item in batch:
        single_mask = torch.tensor([float('-inf')]*max_L)  # masked is True
        true_L = item[1].shape[0]
        single_mask[:true_L] = 0.0    # not masked is False
        padding_mask.append(single_mask)
    padding_mask = torch.stack(padding_mask)   # (b,L) L is the largest length in this batch   (0,-inf)

    
    return input_ids_tensor_train, batch_label_index, padding_mask     #  (b,4,512) (b,L) (b,L)  L is the largest length in this batch 


train_dataset = Dataset_transformer(input_ids_tensor_train, train_label_index_list)
validation_dataset = Dataset_transformer(input_ids_tensor_validation, validation_label_index_list)
batch_size_transformer = args.batch_size_transformer
num_workers = args.num_workers
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size_transformer, shuffle=True, drop_last=True, num_workers=num_workers, collate_fn=collate_fn, pin_memory=True)  
validation_dataloader = DataLoader(dataset=validation_dataset, batch_size=batch_size_transformer, shuffle=False, drop_last=False, num_workers=num_workers, collate_fn=collate_fn, pin_memory=True)
decoder_num_layers = args.decoder_num_layers
transformer_dropout = args.transformer_dropout
transformer_nhead = args.transformer_nhead

In [ ]:
class Transformer_Model(torch.nn.Module):
    def __init__(self):
        super(Transformer_Model, self).__init__()

        self.class_embeddings_start = torch.nn.Parameter(torch.randn(768))
        self.class_embeddings_end_start = torch.nn.Parameter(torch.randn(768))

        self.decoder_layer = torch.nn.TransformerDecoderLayer(d_model=768, nhead=transformer_nhead, dropout=transformer_dropout, activation='gelu', batch_first=True)
        self.transformer_decoder = torch.nn.TransformerDecoder(self.decoder_layer, num_layers=decoder_num_layers)  # forward padding mask(N,L)  attention causal mask(L,L)  multiheadattention  is_causal

        self.linear1 = torch.nn.Linear(768, 768)
        self.gelu = torch.nn.GELU()
        self.linear2 = torch.nn.Linear(768, 768)
        self.softmax = torch.nn.Softmax()

    def forward(self, class_embeddings_dict, mask_dict, batch_text_embedding_tensor, batch_label_index, attention_mask, batch_padding_mask):   #  (b,4,768), (b,L)  (b,L)

        class_embeddings_dict = functions.add_end_classes_embeddings(class_embeddings_dict, self.class_embeddings_start, self.class_embeddings_end_start, tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease, device)
        class_embedding_list = []
        for item in list(mask_dict.keys())[:-1]:
            class_embedding_list.append(class_embeddings_dict[item])
        class_embeddings_tensor = torch.stack(class_embedding_list)    # (C,768) C:17391

        label_embedding_tensor = functions.convert_sequencial_label_to_embedding_tensor(batch_label_index, class_embeddings_dict, index_to_class_label_dict)  # (b,L) --> (b,L,768)
        decoder_output = self.transformer_decoder(label_embedding_tensor, batch_text_embedding_tensor, tgt_mask=attention_mask , tgt_key_padding_mask=batch_padding_mask)#  (b,L,768)  (b,4,768) -->  (b,L,768)

        transformer_output = self.linear1(decoder_output)   #  (b,L,768)
        transformer_output = self.gelu(transformer_output)
        transformer_output = self.linear2(transformer_output)  #  (b,L,768)

        transformer_output = torch.nn.functional.normalize(transformer_output, dim=-1)  # (b,L,768)
        class_embeddings_tensor = torch.nn.functional.normalize(class_embeddings_tensor, dim=-1)  #  (C,768)

        # score_tensor = torch.nn.functional.cosine_similarity(transformer_output.unsqueeze(0).permute((1,0,2)), class_embeddings_tensor.unsqueeze(0), dim=-1)  # (L,1,768) (1,C,768) --> (L,C)
        score_tensor = torch.matmul(transformer_output, class_embeddings_tensor.T)  # (b,L,768), (768,C)  -->  (b,L,C)
        batch_mask_list = []
        for item in batch_label_index:  # (b,L)
            batch_mask_list.append(torch.stack([mask_dict[index_to_class_label_dict[index.item()]] for index in item]))   # (L,C)
        batch_mask_tensor = torch.stack(batch_mask_list)   # (b,L,C)
        batch_mask_tensor = batch_mask_tensor.to(device)
        logits = (score_tensor+batch_mask_tensor)   # (b,L,C)
        return logits  # (b,L,C)
        

In [ ]:
transformer_model = Transformer_Model()
transformer_model = transformer_model.to(device)
learning_rate = args.learning_rate
criterion = torch.nn.CrossEntropyLoss(reduction='sum', ignore_index=17391)    # when compute, the size must be (N, C, L) and (N, L)
optimizer = torch.optim.AdamW(transformer_model.parameters(), lr=learning_rate, weight_decay=args.adam_weight_decay)

In [ ]:
def valid(validation_dataloader):
    transformer_model.eval()
    print('Validation start...')
    with torch.no_grad():
        validation_loss = 0
        category_accuracy = 0
        first_class_accuracy = 0
        second_class_accuracy = 0
        disease_accuracy = 0
        label_f1_0_1_tensor = torch.tensor([])
        predicted_f1_0_1_tensor = torch.tensor([])
        label_AUROC_tesnor = torch.tensor([],dtype=int).to(device)
        predicted_AUROC_tensor = torch.tensor([]).to(device)
        random_outout_index = random.randint(0,1000)
        for i, (batch_input_ids_tensor_valid, batch_label_index, batch_padding_mask) in tqdm(enumerate(validation_dataloader)):

            batch_input_ids_tensor_valid = batch_input_ids_tensor_valid.to(device, non_blocking=True)
            batch_label_index = batch_label_index.to(device, non_blocking=True)
            batch_padding_mask = batch_padding_mask.to(device, non_blocking=True)

            batch_text_embedding_tensor_valid = torch.stack([clinical_Bert_Model(single_input_ids_tensor_valid) for single_input_ids_tensor_valid in batch_input_ids_tensor_valid])  # (b,4,768)
            attention_mask_valid = torch.triu(torch.full((batch_padding_mask.shape[1], batch_padding_mask.shape[1]), True), diagonal=1).to(device, non_blocking=True)   # masked is True, not masked is False
            
            batch_logits = transformer_model(class_embeddings_dict, mask_dict, batch_text_embedding_tensor=batch_text_embedding_tensor_valid, batch_label_index=batch_label_index, attention_mask=attention_mask_valid, batch_padding_mask=batch_padding_mask)   # (b,4,768), (b,L) --> (b,L,C)
            batch_logits = batch_logits.permute((0,2,1))    # --> (b,C,L)
            end_start_index_label = class_label_index_dict['/start'] 
            lable_for_loss_tensor = functions.create_lable_for_loss_tensor(batch_label_index, end_start_index_label, device) # (b,L)
            validation_loss += criterion(batch_logits, lable_for_loss_tensor)

            for j in range(batch_input_ids_tensor_valid.shape[0]):
                
                single_text_embedding_tensor_valid = batch_text_embedding_tensor_valid[j].unsqueeze(0)   # (1,4,768)

                single_label_index_list = batch_label_index[j].tolist()    # [L]
                padding_index = single_label_index_list.index(17391) if 17391 in single_label_index_list else len(single_label_index_list)
                single_label_index_list = single_label_index_list[:padding_index]
                single_sequencial_label = [index_to_class_label_dict[index] for index in single_label_index_list]

                # ingle_label_index_tensor = label_index_list[j].unsqueeze(0)    # (1,L)
                # single_text_embedding_tensor = single_text_embedding_tensor.to(device)
                # single_label_index_tensor = single_label_index_tensor.to(device)

                predicted_index_list = []
                predicted_token_list = []
                predicted_token_probabiluty_list = []
                input_token_index_list = [class_label_index_dict['start']]
                end_start_index_label = class_label_index_dict['/start'] 
                while input_token_index_list[-1] != end_start_index_label:
                    logits = transformer_model(class_embeddings_dict, mask_dict, batch_text_embedding_tensor=single_text_embedding_tensor_valid, batch_label_index=torch.tensor([input_token_index_list]).to(device, non_blocking=True), attention_mask=None, batch_padding_mask=None)   # (1,4,768), (1,L) --> (1,L,C)
                    predicted_index = torch.argmax(logits.squeeze(0), dim=-1)[-1].item()   # (1,L,C)-->int
                    predicted_token_probabiluty_list.append(logits.squeeze(0)[-1])  # (1,L,C) -->[L,(C)]
                    predicted_index_list.append(predicted_index)   # [L]
                    predicted_token_list.append(index_to_class_label_dict[predicted_index])  # [L]
                    input_token_index_list.append(predicted_index)   # [L]

                    if len(predicted_token_list) > 150:
                        break

                predicted_token_probabiluty_tensor = torch.nn.Softmax(dim=-1)(torch.stack(predicted_token_probabiluty_list))  # (L,C)
                predicted_category_tokens_list, predicted_first_class_tokens_list, predicted_second_class_tokens_list, predicted_disease_tokens_list = functions.split_predicted_class_tokens(predicted_token_list, tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease)
                label_category_tokens_list, label_first_class_tokens_list, label_second_class_tokens_list, label_disease_tokens_list = functions.split_predicted_class_tokens(single_sequencial_label, tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease)
                category_accuracy += (len(set(predicted_category_tokens_list).intersection(set(label_category_tokens_list))) / max(len(predicted_category_tokens_list), len(label_category_tokens_list)))
                first_class_accuracy += (len(set(predicted_first_class_tokens_list).intersection(set(label_first_class_tokens_list))) / max(len(predicted_first_class_tokens_list), len(label_first_class_tokens_list)))
                second_class_accuracy += (len(set(predicted_second_class_tokens_list).intersection(set(label_second_class_tokens_list))) / max(len(predicted_second_class_tokens_list), len(label_second_class_tokens_list)))
                disease_accuracy += (len(set(predicted_disease_tokens_list).intersection(set(label_disease_tokens_list))) / max(len(predicted_disease_tokens_list), len(label_disease_tokens_list)))

                single_label_f1_0_1_tensor, single_predicted_f1_0_1_tensor = functions.create_0_1_tensor_for_f1(single_sequencial_label, predicted_disease_tokens_list)
                label_f1_0_1_tensor = torch.cat([label_f1_0_1_tensor, single_label_f1_0_1_tensor])
                predicted_f1_0_1_tensor = torch.cat([predicted_f1_0_1_tensor, single_predicted_f1_0_1_tensor])

                single_probability_AUROC_predicted, single_label_AUROC_0_1_tensor = functions.create_probability_tensor_for_AUROC(single_sequencial_label, class_label_index_dict, predicted_token_probabiluty_tensor, predicted_index_list, device)
                label_AUROC_tesnor = torch.cat([label_AUROC_tesnor, single_label_AUROC_0_1_tensor.to(device).type(torch.int)])
                predicted_AUROC_tensor = torch.cat([predicted_AUROC_tensor, single_probability_AUROC_predicted])

                if 8*i+j == random_outout_index:
                    print('label: ', single_sequencial_label)
                    print('predicted: ', predicted_token_list)



        label_f1_0_1_tensor = label_f1_0_1_tensor.reshape((df_validation_top500.shape[0], 14567))  # (N, C)
        predicted_f1_0_1_tensor = predicted_f1_0_1_tensor.reshape((df_validation_top500.shape[0], 14567))  # (N, C)
        valid_f1_micro = torchmetrics.functional.classification.multilabel_f1_score(predicted_f1_0_1_tensor, label_f1_0_1_tensor, num_labels=14567, threshold=0.5, average='micro', multidim_average='global')
        valid_f1_weighted = torchmetrics.functional.classification.multilabel_f1_score(predicted_f1_0_1_tensor, label_f1_0_1_tensor, num_labels=14567, threshold=0.5, average='weighted', multidim_average='global')


        label_AUROC_tesnor = label_AUROC_tesnor.reshape((df_validation_top500.shape[0], 17391))  # (N, C)
        predicted_AUROC_tensor = predicted_AUROC_tensor.reshape((df_validation_top500.shape[0], 17391))  # (N, C)
        valid_AUROC_micro = torchmetrics.functional.classification.multilabel_auroc(predicted_AUROC_tensor, label_AUROC_tesnor, num_labels=17391, average='micro')
        valid_AUROC_weighted = torchmetrics.functional.classification.multilabel_auroc(predicted_AUROC_tensor, label_AUROC_tesnor, num_labels=17391, average='weighted')

        valid_precision_micro = torchmetrics.functional.classification.multilabel_precision(predicted_f1_0_1_tensor, label_f1_0_1_tensor, num_labels=14567, multidim_average='global', average='micro')
        valid_recall_micro = torchmetrics.functional.classification.multilabel_recall(predicted_f1_0_1_tensor, label_f1_0_1_tensor, num_labels=14567, multidim_average='global', average='micro')

    return validation_loss/df_validation_top500.shape[0], category_accuracy/df_validation_top500.shape[0], first_class_accuracy/df_validation_top500.shape[0], second_class_accuracy/df_validation_top500.shape[0], disease_accuracy/df_validation_top500.shape[0], valid_f1_micro, valid_f1_weighted, valid_AUROC_micro, valid_AUROC_weighted, valid_precision_micro, valid_recall_micro





def train(train_dataloader, validation_dataloader, epochs=5):

    # config = { 'batch_size': batch_size_transformer, 'lr':learning_rate, 'loss':'CrossEntropyLoss', 'optim':'Adam', 
    #           'transformer_decoder_nhead':8, 'transformer_decoder_layers':decoder_num_layers, 'linear_layer':'768-768-Gelu-768', 'epochs':epochs}
    # wandb.init(project="Result", entity="htc-mimic3", config=config)

    print('Training start...')

    for epoch in range(epochs):
        print(f'Epoch: {epoch+1}')
        loss_training_epoch = 0

        transformer_model.train()
        for i, (batch_input_ids_tensor_train, batch_label_index, batch_padding_mask) in tqdm(enumerate(train_dataloader)):
            loss_training_batch = 0

            batch_input_ids_tensor_train = batch_input_ids_tensor_train.to(device, non_blocking=True)   #  (b,4,512)
            batch_label_index = batch_label_index.to(device, non_blocking=True)  # (b,L)  L is the largest length in this batch 
            batch_padding_mask = batch_padding_mask.to(device, non_blocking=True)  # (b,L)  L is the largest length in this batch 

            batch_text_embedding_tensor = torch.stack([clinical_Bert_Model(single_input_ids_tensor_train) for single_input_ids_tensor_train in batch_input_ids_tensor_train])
            # (b,4,768)
            attention_mask_train = torch.triu(torch.full((batch_padding_mask.shape[1], batch_padding_mask.shape[1]), True), diagonal=1).to(device, non_blocking=True)
             # masked is True, not masked is False
            batch_logits = transformer_model(class_embeddings_dict, mask_dict, batch_text_embedding_tensor=batch_text_embedding_tensor, batch_label_index=batch_label_index, attention_mask=attention_mask_train, batch_padding_mask=batch_padding_mask)   # (b,4,768), (b,L) --> (b,L,C)
            batch_logits = batch_logits.permute((0,2,1))    # --> (b,C,L)
            end_start_index_label = class_label_index_dict['/start'] 
            lable_for_loss_tensor = functions.create_lable_for_loss_tensor(batch_label_index, end_start_index_label, device) # (b,L)
            loss_training_batch = criterion(batch_logits, lable_for_loss_tensor)  # (b,C,L), (b,L) --> tensor

            optimizer.zero_grad()
            loss_training_batch.backward()
            optimizer.step()         
            loss_training_epoch += loss_training_batch.item()

            

        
        print('the training loss in epoch',epoch+1,'is:',loss_training_epoch/df_train_top500.shape[0])
        validation_loss, category_accuracy, first_class_accuracy, second_class_accuracy, disease_accuracy, valid_f1_micro, valid_f1_weighted, valid_AUROC_micro, valid_AUROC_weighted, valid_precision_micro, valid_recall_micro= valid(validation_dataloader)
        # wandb.log({'training_loss': loss_training_epoch/df_train_top500.shape[0]})
        # wandb.log({'validation_loss': validation_loss.item()})
        # wandb.log({'valid_category_accuracy': category_accuracy})
        # wandb.log({'valid_first_class_accuracy': first_class_accuracy})
        # wandb.log({'valid_second_class_accuracy': second_class_accuracy})
        # wandb.log({'valid_disease_accuracy': disease_accuracy})
        # wandb.log({'valid_f1_micro': valid_f1_micro.item()})
        # wandb.log({'valid_f1_weighted': valid_f1_weighted.item()})
        # wandb.log({'valid_AUROC_micro': valid_AUROC_micro.item()})
        # wandb.log({'valid_AUROC_weighted': valid_AUROC_weighted.item()})
        # wandb.log({'valid_precision_micro': valid_precision_micro.item()})
        # wandb.log({'valid_recall_micro': valid_recall_micro.item()})

        print('the valid loss in epoch',epoch+1,'is:', validation_loss.item())
        print('the valid accuracy in epoch',epoch+1,'is:', category_accuracy, first_class_accuracy, second_class_accuracy, disease_accuracy)
        print('the valid f1 micro in epoch',epoch+1,'is:', valid_f1_micro.item())
        print('the valid f1 weighted in epoch',epoch+1,'is:', valid_f1_weighted.item())
        print('the valid AUROC micro in epoch',epoch+1,'is:', valid_AUROC_micro.item())
        print('the valid AUROC weighted in epoch',epoch+1,'is:', valid_AUROC_weighted.item())
        print('the valid_precision micro in epoch',epoch+1,'is:', valid_precision_micro.item())
        print('the valid_recall_micro weighted in epoch',epoch+1,'is:', valid_recall_micro.item())
    
    print('Training Finished')



In [ ]:
# mp.set_start_method('spawn',force=True)
train(train_dataloader, validation_dataloader, epochs=args.epoch)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ranniu (htc-mimic3). Use `wandb login --relogin` to force relogin


Training start...
Epoch: 1


1062it [25:52,  1.46s/it]

the training loss in epoch 1 is: 90.04556358466714
Validation start...



188it [17:57,  5.73s/it]
/u/home/niur/venv_1/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
/u/home/niur/venv_1/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


...

the valid loss in epoch 29 is: 83.4811019897461
the valid accuracy in epoch 29 is: 0.46533261738261733 0.3388451915129926 0.2105817112597293 0.13810578147044
the valid f1 micro in epoch 29 is: 0.20360900461673737
the valid f1 weighted in epoch 29 is: 0.13561096787452698
the valid AUROC micro in epoch 29 is: 0.6303483843803406
the valid AUROC weighted in epoch 29 is: 0.5784757137298584
Epoch: 30


1062it [23:55,  1.35s/it]

the training loss in epoch 30 is: 84.88639604888169
Validation start...



188it [27:19,  8.72s/it]
/u/home/niur/venv_1/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)
/u/home/niur/venv_1/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


the valid loss in epoch 30 is: 83.40372467041016
the valid accuracy in epoch 30 is: 0.42448636178636223 0.2940718078991288 0.1936086626307985 0.12380353206678746
the valid f1 micro in epoch 30 is: 0.18772165477275848
the valid f1 weighted in epoch 30 is: 0.13539527356624603
the valid AUROC micro in epoch 30 is: 0.6181236505508423
the valid AUROC weighted in epoch 30 is: 0.5746471285820007
Training Finished


In [ ]:
# click  argparse